<a href="https://colab.research.google.com/github/joaoflauzino/cafe_c_codigo_01/blob/main/cafe_c_codigo_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 41.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=f8607ecad39c31e9305dbabe56c8604756aaeaffbb5a024997de7231826a6d00
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


# Bibliotecas

In [ ]:
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Criando sessão no spark

In [ ]:
## Criando sessão
spark = SparkSession.builder.appName("NBO").getOrCreate()

# Leitura de dados

In [ ]:
## Leitura de csv 
df = spark.read.csv('/content/drive/My Drive/Datasets/marketing_campaign.csv', sep = ";", header=True)

# Informações sobre a base de dados

- AcceptedCmp1 - 1 if customer accepted the offer in the 1st campaign, 0 otherwise
- AcceptedCmp2 - 1 if customer accepted the offer in the 2nd campaign, 0 otherwise
- AcceptedCmp3 - 1 if customer accepted the offer in the 3rd campaign, 0 - otherwise
- AcceptedCmp4 - 1 if customer accepted the offer in the 4th campaign, 0 otherwise
- AcceptedCmp5 - 1 if customer accepted the offer in the 5th campaign, 0 otherwise
- Response (target) - 1 if customer accepted the offer in the last campaign, 0 otherwise
- Complain - 1 if customer complained in the last 2 years
- DtCustomer - date of customer’s enrolment with the company
- Education - customer’s level of education
- Marital - customer’s marital status
- Kidhome - number of small children in customer’s household
- Teenhome - number of teenagers in customer’s household
- Income - customer’s yearly household income
- MntFishProducts - amount spent on fish products in the last 2 years
- MntMeatProducts - amount spent on meat products in the last 2 years
- MntFruits - amount spent on fruits products in the last 2 years
- MntSweetProducts - amount spent on sweet products in the last 2 years
- MntWines - amount spent on wine products in the last 2 years
- MntGoldProds - amount spent on gold products in the last 2 years
- NumDealsPurchases - number of purchases made with discount
- NumCatalogPurchases - number of purchases made using catalogue
- NumStorePurchases - number of purchases made directly in stores
- NumWebPurchases - number of purchases made through company’s web site
- NumWebVisitsMonth - number of visits to company’s web site in the last month
- Recency - number of days since the last purchase



```
# This is formatted as code
```

# Perguntas sobre o negócio

- Daqueles que são propensos a aceitar o produto, qual o nível de graduação mais presente? O nível de graduação difere daqueles que aceitam ou não uma oferta?

- Daqueles que são propensos a aceitar o produto, qual o Estado Civil mais presente? O Estado Civil difere daqueles que aceitam ou não uma oferta?

- Quanto mais recente, maior a chance dele aceitar a oferta?

- Quanto maior a renda, maior a chance dele aceitar a oferta?

- Existe alguma correlação entre renda e recência?

- Pessoas mais velhas tendem a aceitar/recusar as ofertas? (criar campo com idade em anos)

- A plataforma que o produto é oferecido impacta na aceitação das ofertas?

- O número de crianças impacta na aceitação de ofertas?

- O número de adolescentes impacta na aceitação de ofertas?
- As campanhas são cumulativas ou independentes? Se for cumulativas, esses clientes reclamaram alguma vez nos últimos 2 anos?

- Clientes mais antigos tendem a aceitar as ofertas? (criar campo com idade em anos)

- Visitas no site da empresa tem correlação com a aceita/recusa das ofertas?


In [24]:
df.show()

+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+------------+------------+------------+------------+------------+--------+-------------+---------+--------+----+
|  ID|Year_Birth| Education|Marital_Status|Income|Kidhome|Teenhome|Dt_Customer|Recency|MntWines|MntFruits|MntMeatProducts|MntFishProducts|MntSweetProducts|MntGoldProds|NumDealsPurchases|NumWebPurchases|NumCatalogPurchases|NumStorePurchases|NumWebVisitsMonth|AcceptedCmp3|AcceptedCmp4|AcceptedCmp5|AcceptedCmp1|AcceptedCmp2|Complain|Z_CostContact|Z_Revenue|Response|FLAG|
+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+

## Tratamento da base